<a href="https://colab.research.google.com/github/dariamm03/Machine_Learning/blob/main/MachineLearning11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание:
Необходимо обучить модель для определения принадлежности текста к одному из заданных классов с использованием Bag of words.
Например, построить модель для определения принадлежности текста определенному автору по части текста.


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

Bag of Words — это модель, часто используемая при обработке текстов, представляющая собой неупорядоченный набор слов, входящих в обрабатываемый текст

In [ ]:
data = pd.read_csv('DataSet13_1.csv', sep=',')
data

,id,text,author
0,0,-Бабушка!- вскричала малютка.- Возьми меня с с...,Dostoevsky
1,1,"Знал ли Скрудж об этом? Разумеется, знал. Да и...",Dostoevsky
2,2,"-С праздником, дядя, с радостью! Дай вам Бог в...",Dostoevsky
3,3,Мы высказали только главную передовую мысль на...,Dostoevsky
4,4,"I. Отдел литературный. Повести, романы, расска...",Dostoevsky
...,...,...,...
1729,1729,"-Хотелось бы мне, чтоб он был здесь.\nКанторек...",Remark
1730,1730,Но для нас в этом-то и заключается их несостоя...,Remark
1731,1731,"-Как дела, Франц?- спрашивает Кропп.\nКеммерих...",Remark
1732,1732,"Мюллер наклоняется:\n-Мы принесли твои вещи, Ф...",Remark


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.drop(['author', 'id'], axis=1), data.author, test_size=0.2, random_state=42)

In [ ]:
#Частота вхождения слова в текст
vectarizer = CountVectorizer()
vectarizer.fit(x_train.append(x_test).text)

<ipython-input-11-af84a9c7ed90>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vectarizer.fit(x_train.append(x_test).text)


CountVectorizer()

In [ ]:
x_train = vectarizer.transform(x_train.text)
x_test = vectarizer.transform(x_test.text)
y_train = y_train.values
y_test = y_test.values

**Обучение классификатора**

In [ ]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred = classifier.predict(x_test)

pres = precision_score(y_test, y_pred, average='weighted')
p = classifier.predict_proba(x_test)[:,1]
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print('precision Bow Score = {0}'.format(pres))
print('recall Bow Score = {0}'.format(recall))
print('F1 Bow Score = {0}'.format(f1))


precision Bow Score = 0.7906572487025652
recall Bow Score = 0.792507204610951
F1 Bow Score = 0.7814049884917302


**Предобработка**

In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
  return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.corpus import stopwords

Stopwords = set(stopwords.words('russian'))
def remove_stopwords(text):
  return " ".join([word for word in str(text).split() if word not in Stopwords])

In [ ]:
import string

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
  return text.translate(str.maketrans('','', PUNCT_TO_REMOVE))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.drop(['author', 'id'], axis=1), data.author, test_size=0.2, random_state=42)

In [ ]:
train_preprocessed = {'text':[], 'author':[]}
train_preprocessed = pd.DataFrame(train_preprocessed)

for index, row in x_train.iterrows():
  new_row = pd.DataFrame({'text': lemmatize_words(remove_stopwords(remove_punctuation(row['text']))), 'author': y_train[index]}, index=[index])
  train_preprocessed = pd.concat([train_preprocessed, new_row]).reset_index(drop=True)

test_preprocessed = {'text':[], 'author':[]}
test_preprocessed = pd.DataFrame(test_preprocessed)

for index, row in x_test.iterrows():
  new_row = pd.DataFrame({'text':lemmatize_words(remove_stopwords(remove_punctuation(row['text']))), 'author': y_test[index]}, index=[index])
  test_preprocessed = pd.concat([test_preprocessed, new_row]).reset_index(drop=True)

In [ ]:
x_train = train_preprocessed.drop(['author'], axis=1)
y_train = train_preprocessed.author
x_test = test_preprocessed.drop(['author'], axis=1)
y_test = test_preprocessed.author

создает матрицу, в которой каждое уникальное слово представлено столбцом матрицы, а каждый образец текста из документа - строкой в матрице. Значение каждой ячейки - количество слов в данном конкретном примере текста.

In [ ]:
vectarizer = CountVectorizer()
vectarizer.fit(x_train.append(x_test).text)

x_train = vectarizer.transform(x_train.text)
x_test = vectarizer.transform(x_test.text)
y_train = y_train.values
y_test = y_test.values

<ipython-input-22-7ba3b58b513f>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vectarizer.fit(x_train.append(x_test).text)


In [ ]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred = classifier.predict(x_test)

pres = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print('precision Bow Score = {0}'.format(pres))
print('recall Bow Score = {0}'.format(recall))
print('F1 Bow Score = {0}'.format(f1))

precision Bow Score = 0.8109539194988599
recall Bow Score = 0.8126801152737753
F1 Bow Score = 0.7987303714087575
